<a href="https://colab.research.google.com/github/FiorelaCiroku/LLM---Akademia-e-Forcave-te-Armatosura/blob/main/Tokenizer_%26_Embeddings_LLMs_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

from transformers import (
    RobertaTokenizerFast,      # BPE
    GPT2TokenizerFast,         # byte-level BPE
    AutoTokenizer,             # WordPiece (mBERT)
    T5Tokenizer,               # SentencePiece (mT5)
)

In [ ]:
"""
Tokenization showcase on an Albanian sentence.

Demonstrates:
1. Character-level
2. Word-level (whitespace)
3. Word-level (punctuation as separate tokens)
4. Rule-based (slightly smarter regex)
5. Toy morpheme-based (Albanian-ish)
6. Subword BPE (RoBERTa)
7. WordPiece (mBERT)
8. SentencePiece / Unigram LM (mT5)
9. Byte-level BPE (GPT-2)
"""

SENTENCE = "Cili institucion paralajmëron popullatën gjatë përmbytjeve?"


# 1️⃣ CHARACTER-LEVEL TOKENIZATION
def char_tokenize(text: str):
    return list(text)


# 2️⃣ WORD-LEVEL (WHITESPACE) TOKENIZATION
def word_whitespace_tokenize(text: str):
    return text.split()


# 3️⃣ WORD-LEVEL (PUNCTUATION SEPARATE) TOKENIZATION
def word_punct_tokenize(text: str):
    # \w+ = word chars, [^\w\s] = any non-word, non-space (punctuation, symbols)
    return re.findall(r"\w+|[^\w\s]", text, flags=re.UNICODE)


# 4️⃣ RULE-BASED TOKENIZATION (slightly smarter)
def rule_based_tokenize(text: str):
    """
    Simple heuristic:
    - Split on whitespace
    - Separate trailing punctuation .,!?;:
    - Keep hyphenated and apostrophe words together
    """
    tokens = []
    for token in text.split():
        # Separate trailing punctuation if present
        m = re.match(r"(.+?)([.,!?;:]+)$", token)
        if m:
            word, punct = m.groups()
            tokens.append(word)
            tokens.append(punct)
        else:
            tokens.append(token)
    return tokens


# 5️⃣ TOY MORPHEME-BASED TOKENIZATION (Albanian-ish)
def toy_morpheme_tokenize(text: str):
    """
    Extremely simplified "morphological" splitter:
    - Split into words (with punctuation)
    - On each word, try stripping a small set of common Albanian suffixes.
    NOTE: This is *illustrative only*, not linguistically correct.
    """
    base_tokens = word_punct_tokenize(text)
    suffixes = [
        "ve", "ve", "të", "t", "ën", "et", "jes", "uar", "uarit",
        "je", "jet", "shëm", "shme"
    ]

    result = []
    for tok in base_tokens:
        # leave punctuation alone
        if re.fullmatch(r"[^\w\s]+", tok, flags=re.UNICODE):
            result.append(tok)
            continue

        lowered = tok.lower()
        split_done = False
        for suf in sorted(suffixes, key=len, reverse=True):
            if lowered.endswith(suf) and len(tok) > len(suf) + 1:
                stem = tok[:len(tok) - len(suf)]
                result.append(stem)
                result.append(tok[len(tok) - len(suf):])
                split_done = True
                break

        if not split_done:
            result.append(tok)

    return result


# 🤖 PREPARE TRANSFORMERS TOKENIZERS
def load_hf_tokenizers():
    # Subword BPE
    roberta_tok = RobertaTokenizerFast.from_pretrained("roberta-base")

    # WordPiece: multilingual BERT
    mbert_tok = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

    # SentencePiece / Unigram LM: mT5
    sp_tok = T5Tokenizer.from_pretrained("google/mt5-small")

    # Byte-level BPE: GPT-2
    gpt2_tok = GPT2TokenizerFast.from_pretrained("gpt2")

    return roberta_tok, mbert_tok, sp_tok, gpt2_tok


def main():
    print("\nSentence:")
    print(SENTENCE)
    print("=" * 80)

    # 1) Character-level
    print("\n1️⃣ Character-level tokenization:")
    print(char_tokenize(SENTENCE))

    # 2) Word-level (whitespace)
    print("\n2️⃣ Word-level (whitespace) tokenization:")
    print(word_whitespace_tokenize(SENTENCE))

    # 3) Word-level (punctuation as separate tokens)
    print("\n3️⃣ Word-level (punctuation separate) tokenization:")
    print(word_punct_tokenize(SENTENCE))

    # 4) Rule-based heuristic tokenization
    print("\n4️⃣ Rule-based heuristic tokenization:")
    print(rule_based_tokenize(SENTENCE))

    # 5) Toy morpheme-based tokenization (Albanian-ish)
    print("\n5️⃣ Toy morpheme-based tokenization (illustrative):")
    print(toy_morpheme_tokenize(SENTENCE))

    # Load Hugging Face tokenizers
    roberta_tok, mbert_tok, sp_tok, gpt2_tok = load_hf_tokenizers()

    # 6) Subword BPE (RoBERTa)
    print("\n6️⃣ Subword BPE tokenization (RoBERTa):")
    print(roberta_tok.tokenize(SENTENCE))

    # 7) WordPiece (BERT multilingual)
    print("\n7️⃣ WordPiece tokenization (mBERT):")
    print(mbert_tok.tokenize(SENTENCE))

    # 8) SentencePiece / Unigram LM (mT5)
    print("\n8️⃣ SentencePiece (Unigram LM) tokenization (mT5):")
    print(sp_tok.tokenize(SENTENCE))

    # 9) Byte-level BPE (GPT-2)
    print("\n9️⃣ Byte-level BPE tokenization (GPT-2):")
    print(gpt2_tok.tokenize(SENTENCE))


if __name__ == "__main__":
    main()



Sentence:
Cili institucion paralajmëron popullatën gjatë përmbytjeve?

1️⃣ Character-level tokenization:
['C', 'i', 'l', 'i', ' ', 'i', 'n', 's', 't', 'i', 't', 'u', 'c', 'i', 'o', 'n', ' ', 'p', 'a', 'r', 'a', 'l', 'a', 'j', 'm', 'ë', 'r', 'o', 'n', ' ', 'p', 'o', 'p', 'u', 'l', 'l', 'a', 't', 'ë', 'n', ' ', 'g', 'j', 'a', 't', 'ë', ' ', 'p', 'ë', 'r', 'm', 'b', 'y', 't', 'j', 'e', 'v', 'e', '?']

2️⃣ Word-level (whitespace) tokenization:
['Cili', 'institucion', 'paralajmëron', 'popullatën', 'gjatë', 'përmbytjeve?']

3️⃣ Word-level (punctuation separate) tokenization:
['Cili', 'institucion', 'paralajmëron', 'popullatën', 'gjatë', 'përmbytjeve', '?']

4️⃣ Rule-based heuristic tokenization:
['Cili', 'institucion', 'paralajmëron', 'popullatën', 'gjatë', 'përmbytjeve', '?']

5️⃣ Toy morpheme-based tokenization (illustrative):
['Cili', 'institucion', 'paralajmëron', 'popullat', 'ën', 'gja', 'të', 'përmbytje', 've', '?']

6️⃣ Subword BPE tokenization (RoBERTa):
['C', 'ili', 'Ġinstit', 'uc'

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

text = "Cili institucion paralajmëron popullatën gjatë përmbytjeve?"
tokens = tokenizer.tokenize(text)
ids = tokenizer.encode(text, add_special_tokens=True)

print("Tokens:", tokens)
print("Token IDs:", ids)

Tokens: ['Ci', '##li', 'ins', '##tit', '##uc', '##ion', 'paral', '##aj', '##më', '##ron', 'pop', '##ulla', '##të', '##n', 'gjatë', 'për', '##mb', '##yt', '##jeve', '?']
Token IDs: [101, 51457, 10390, 15498, 23516, 43805, 11046, 102568, 21171, 68001, 11620, 13717, 63486, 24205, 10115, 38065, 12329, 27012, 20246, 53208, 136, 102]


In [ ]:
import numpy as np
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer


# 1. TOKENIZE 10 ALBANIAN EMERGENCY TERMS
def tokenize_emergency_terms(sp_tokenizer, emb_model):
    emergency_terms = [
        "përmbytje",
        "tërmet",
        "zjarr",
        "evakuim",
        "ndihmë",
        "ambulancë",
        "urgjencë",
        "stuhi",
        "rrëshqitje dheu",
        "shpëtim"
    ]

    print("\n=== 1) Tokenize 10 Albanian emergency terms ===\n")
    for term in emergency_terms:
        # SentencePiece-style tokens
        tokens = sp_tokenizer.tokenize(term)
        # Token IDs
        ids = sp_tokenizer.encode(term, add_special_tokens=False)

        # Embedding with MiniLM (sentence-transformers)
        emb = emb_model.encode(term, normalize_embeddings=True)

        print(f"Term: {term}")
        print(f"  Tokens: {tokens}")
        print(f"  Token IDs: {ids}")
        print(f"  Embedding (first 384 dims): {emb[:384]}\n")  # avoid printing 384 dims fully


# 2. CREATE A ~200-WORD ALBANIAN PARAGRAPH AND CHUNK IT
def build_albanian_paragraph():
    text = (
        "Njoftoni popullatën për përmbytje dhe rrezik të menjëhershëm. "
        "Autoritetet lokale po paralajmërojnë banorët që të qëndrojnë larg brigjeve të lumit "
        "dhe të mos përdorin urat e vjetra. "
        "Shërbimet e emergjencës janë në gjendje gatishmërie dhe ekipet e shpëtimit po kontrollojnë zonat më të rrezikuara. "
        "Familjet që jetojnë pranë lumit janë udhëzuar të përgatisin çanta me gjërat më të domosdoshme, "
        "si ujë, ushqim, rroba të ngrohta, dokumente personale dhe ilaçe. "
        "Në rast evakuimi, banorët duhet të ndjekin udhëzimet e policisë dhe të mos kthehen në shtëpi pa një njoftim zyrtar. "
        "Qendra e emergjencës ka hapur një linjë telefonike për informacione shtesë dhe raportim incidentesh. "
        "Për të shmangur panikun, qytetarët këshillohen të ndjekin vetëm burimet zyrtare të informacionit dhe të mos shpërndajnë lajme të pakonfirmuara. "
        "Në shkolla dhe institucione publike po zhvillohen ushtrime evakuimi për t'i mësuar njerëzit si të veprojnë gjatë një përmbytjeje të madhe. "
        "Mjekët dhe infermierët janë në gatishmëri për të ofruar ndihmë mjekësore nëse ka të lënduar. "
        "Njësitë e zjarrfikësve po kontrollojnë gjithashtu për rrëshqitje dheu në zonat malore, "
        "sepse reshjet e dendura mund të shkaktojnë bllokim rrugësh dhe izolim të fshatrave. "
        "Banorëve u kujtohet të mos e rrezikojnë veten duke kaluar me makinë nëpër rrugë të mbuluara nga uji "
        "dhe të mos tentojnë të kalojnë lumin në këmbë. "
        "Nëse uji hyn në ndërtesa, energjia elektrike duhet të fiket menjëherë për të shmangur rrezikun e goditjes nga rryma. "
        "Qytetarët mund të strehohen në qendrat komunitare të përgatitura posaçërisht për emergjenca, "
        "ku ofrohen batanije, ushqim i ngrohtë dhe asistencë psikologjike. "
        "Pas kalimit të rrezikut, ekspertët do të vlerësojnë dëmet në infrastrukturë dhe në banesa, "
        "dhe autoritetet do të njoftojnë për hapat e mëtejshëm të rindërtimit."
    )

    words = text.split()
    words_200 = words[:200]
    paragraph_200 = " ".join(words_200)
    return paragraph_200


def chunk_text_by_words(text, max_words=50):
    """Chunk text into pieces of up to max_words, preserving order."""
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunk_words = words[i:i + max_words]
        chunks.append(" ".join(chunk_words))
    return chunks


# 3. COMPUTE EMBEDDINGS WITH MiniLM AND COMPARE SIMILARITIES
def compute_minilm_embeddings(chunks, emb_model):
    print("\n=== 3) Compute embeddings with MiniLM and compare similarity scores ===\n")

    embeddings = emb_model.encode(chunks, normalize_embeddings=True)
    sim_matrix = np.matmul(embeddings, embeddings.T)

    print("Chunks:")
    for i, ch in enumerate(chunks):
        print(f"\nChunk {i} (len={len(ch.split())} words):")
        print(ch)

    print("\nSimilarity matrix (cosine):")
    np.set_printoptions(precision=3, suppress=True)
    print(sim_matrix)

    return embeddings, sim_matrix


# 4. IDENTIFY ONE "BAD CHUNK" AND FIX IT
def find_bad_chunk(sim_matrix):
    """Identify the chunk with the lowest average similarity to other chunks."""
    n = sim_matrix.shape[0]
    avg_sims = []
    for i in range(n):
        avg_sim = (sim_matrix[i].sum() - 1.0) / (n - 1)  # exclude self-similarity
        avg_sims.append(avg_sim)

    bad_index = int(np.argmin(avg_sims))
    return bad_index, avg_sims


def fix_bad_chunk(chunks, sim_matrix):
    print("\n=== 4) Identify one 'bad chunk' and fix it ===\n")

    bad_idx, avg_sims = find_bad_chunk(sim_matrix)

    print("Average similarity per chunk:")
    for i, s in enumerate(avg_sims):
        print(f"Chunk {i}: avg similarity = {s:.3f}")

    print(f"\n👉 Bad chunk candidate: Chunk {bad_idx} (lowest avg similarity)\n")
    print("Bad chunk content:")
    print(chunks[bad_idx], "\n")

    n = len(chunks)
    if n == 1:
        print("Only one chunk, nothing to fix.")
        return chunks

    # Choose neighbor with higher similarity
    if bad_idx == 0:
        neighbor_idx = 1
    elif bad_idx == n - 1:
        neighbor_idx = n - 2
    else:
        sim_prev = sim_matrix[bad_idx, bad_idx - 1]
        sim_next = sim_matrix[bad_idx, bad_idx + 1]
        neighbor_idx = bad_idx - 1 if sim_prev >= sim_next else bad_idx + 1

    print(f"Merging bad chunk {bad_idx} with neighbor chunk {neighbor_idx} to improve coherence.\n")

    # Build merged list
    new_chunks = []
    i = 0
    while i < n:
        if i == bad_idx:
            if neighbor_idx < bad_idx:
                merged_text = chunks[neighbor_idx] + " " + chunks[bad_idx]
                new_chunks.append(merged_text)
                i += 1  # neighbor already handled earlier
            else:
                merged_text = chunks[bad_idx] + " " + chunks[neighbor_idx]
                new_chunks.append(merged_text)
                i += 2  # skip bad + neighbor
        elif i == neighbor_idx:
            i += 1  # will be handled in bad_idx branch
        else:
            new_chunks.append(chunks[i])
            i += 1

    print("New chunks after fix:")
    for j, ch in enumerate(new_chunks):
        print(f"\nChunk {j} (len={len(ch.split())} words):")
        print(ch)

    return new_chunks


def main():
    # Tokenizer for Albanian (multilingual SentencePiece — mT5)
    sp_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

    # Embedding model (MiniLM)
    emb_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    # 1) Tokenize 10 emergency terms, print tokens, IDs, and embeddings
    tokenize_emergency_terms(sp_tokenizer, emb_model)

    # 2) Build a ~200-word paragraph and chunk it
    print("\n=== 2) Build a ~200-word paragraph and break into chunks ===\n")
    paragraph_200 = build_albanian_paragraph()
    print("Paragraph (~200 words):")
    print(paragraph_200)

    chunks = chunk_text_by_words(paragraph_200, max_words=50)
    print("\nChunks created (word-based, 50 words each):")
    for i, ch in enumerate(chunks):
        print(f"\nChunk {i} (len={len(ch.split())} words):")
        print(ch)

    # 3) Compute embeddings with MiniLM and compare similarity scores
    embeddings, sim_matrix = compute_minilm_embeddings(chunks, emb_model)

    # 4) Identify one “bad chunk” and fix it
    fixed_chunks = fix_bad_chunk(chunks, sim_matrix)

    # 5) (Optional) Recompute similarity after fix
    print("\n=== 5) Recompute similarity after fixing the bad chunk (optional) ===\n")
    if fixed_chunks:
        fixed_embeddings = emb_model.encode(fixed_chunks, normalize_embeddings=True)
        fixed_sim_matrix = np.matmul(fixed_embeddings, fixed_embeddings.T)
        print("New similarity matrix after fix:")
        print(fixed_sim_matrix)


if __name__ == "__main__":
    main()



=== 1) Tokenize 10 Albanian emergency terms ===

Term: përmbytje
  Tokens: ['▁për', 'mby', 'tje']
  Token IDs: [1212, 85107, 10281]
  Embedding (first 384 dims): [ 0.017  0.06  -0.065 -0.009 -0.067  0.028  0.1    0.062  0.081 -0.023
  0.008 -0.033 -0.042  0.005 -0.045  0.054  0.022  0.049 -0.069 -0.007
 -0.097 -0.035  0.044 -0.091  0.063  0.038  0.015  0.114  0.039 -0.098
  0.061 -0.003  0.068 -0.022  0.004  0.058 -0.041  0.004 -0.026  0.007
 -0.044 -0.041 -0.092  0.032 -0.038 -0.011  0.013  0.019 -0.02  -0.088
 -0.072 -0.097 -0.039 -0.069  0.009 -0.055  0.002  0.047 -0.003 -0.044
  0.002 -0.011 -0.024  0.018 -0.005 -0.051  0.058  0.057 -0.036  0.007
  0.091  0.016 -0.032  0.002 -0.014  0.017 -0.023 -0.037  0.024 -0.053
 -0.008  0.002 -0.038  0.012 -0.021  0.001  0.018  0.002  0.008 -0.03
 -0.012 -0.006 -0.084  0.001 -0.089  0.072  0.03  -0.043 -0.014  0.103
  0.044  0.043  0.053  0.053  0.053 -0.017  0.033 -0.031  0.036 -0.029
 -0.077 -0.051  0.003  0.055  0.024  0.011  0.062  0.055 

In [ ]:
!pip install sentencepiece
import sentencepiece as spm


# ==== CONFIGURATION SECTION (edit these paths only) ====
INPUT_CORPUS = "/content/Corpus shqip.rtf"   # <--- INSERTED PATH HERE
MODEL_PREFIX = "albanian_sp"
VOCAB_SIZE = 629
MODEL_TYPE = "unigram"  # can be: unigram | bpe | word | char
# =======================================================


def train_tokenizer():
    """
    Train a SentencePiece tokenizer for Albanian
    using the constants defined above.
    """

    spm.SentencePieceTrainer.train(
        input=INPUT_CORPUS,
        model_prefix=MODEL_PREFIX,
        vocab_size=VOCAB_SIZE,
        model_type=MODEL_TYPE,
        character_coverage=1.0,
        input_sentence_size=2000000,
        shuffle_input_sentence=True,
        pad_id=0,
        unk_id=1,
        bos_id=2,
        eos_id=3,
    )

    print("✅ Tokenizer training complete.")
    print(f"Created: {MODEL_PREFIX}.model and {MODEL_PREFIX}.vocab")


def demo_tokenization():
    """
    Demonstrate the trained tokenizer.
    """

    sp = spm.SentencePieceProcessor()
    sp.load(f"{MODEL_PREFIX}.model")

    sentence = "Njoftoni popullatën për përmbytje."
    tokens = sp.encode(sentence, out_type=str)
    ids = sp.encode(sentence, out_type=int)

    print("\n🔹 Sentence:", sentence)
    print("🔹 Tokens:", tokens)
    print("🔹 IDs:", ids)
    print("🔹 Decoded:", sp.decode(ids))


if __name__ == "__main__":
    train_tokenizer()
    demo_tokenization()


✅ Tokenizer training complete.
Created: albanian_sp.model and albanian_sp.vocab

🔹 Sentence: Njoftoni popullatën për përmbytje.
🔹 Tokens: ['▁N', 'joft', 'oni', '▁popull', 'at', 'ë', 'n', '▁p', 'ë', 'r', '▁p', 'ë', 'r', 'mby', 'tje', '.']
🔹 IDs: [22, 267, 412, 255, 32, 1, 8, 10, 1, 29, 10, 1, 29, 574, 521, 7]
🔹 Decoded: Njoftoni popullat ⁇ n p ⁇ r p ⁇ rmbytje.


In [ ]:
import re
import numpy as np
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer


DOC_PATH = "/content/Mbrojtja civile manuali.rtf"  # <- your full Albanian emergency document


# 1. TOKENIZE 10 ALBANIAN EMERGENCY TERMS
def tokenize_emergency_terms(sp_tokenizer, emb_model):
    emergency_terms = [
        "përmbytje",
        "tërmet",
        "zjarr",
        "evakuim",
        "ndihmë",
        "ambulancë",
        "urgjencë",
        "stuhi",
        "rrëshqitje dheu",
        "shpëtim"
    ]

    print("\n=== 1) Tokenize 10 Albanian emergency terms ===\n")
    for term in emergency_terms:
        # SentencePiece-style tokens (from mT5 tokenizer)
        tokens = sp_tokenizer.tokenize(term)
        # Token IDs (no special tokens)
        ids = sp_tokenizer.encode(term, add_special_tokens=False)
        # Embedding with MiniLM (sentence-transformers)
        emb = emb_model.encode(term, normalize_embeddings=True)

        print(f"Term: {term}")
        print(f"  Tokens: {tokens}")
        print(f"  Token IDs: {ids}")
        print(f"  Embedding (first 10 dims): {emb[:10]}\n")  # only show first 10 dims


def load_emergency_document(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
    return text


# --- NEW: sentence-based splitting and chunking --- #

def split_into_sentences(text: str):
    """
    Very simple sentence splitter based on punctuation.
    Works reasonably well for Albanian prose, but is not perfect.
    """
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    # Split on ., !, ? followed by space and a capital letter or digit
    # Keep the delimiter by using a capturing group.
    parts = re.split(r"([.!?])\s+", text)

    sentences = []
    current = ""
    for i in range(0, len(parts), 2):
        if i >= len(parts):
            break
        chunk = parts[i].strip()
        punct = parts[i + 1] if i + 1 < len(parts) else ""

        if not chunk:
            continue

        sentence = (chunk + punct).strip()
        if sentence:
            sentences.append(sentence)

    return sentences


def chunk_text_by_sentences(text: str, max_words_per_chunk: int = 80):
    """
    Build chunks as *groups of full sentences*, such that each chunk
    has at most `max_words_per_chunk` words.

    This preserves sentence boundaries while still controlling chunk size.
    """
    sentences = split_into_sentences(text)
    chunks = []
    current_chunk = []
    current_word_count = 0

    for sent in sentences:
        sent_word_count = len(sent.split())

        # If adding this sentence would exceed the limit, start a new chunk
        if current_chunk and current_word_count + sent_word_count > max_words_per_chunk:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sent]
            current_word_count = sent_word_count
        else:
            current_chunk.append(sent)
            current_word_count += sent_word_count

    # Last chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


# 3. COMPUTE EMBEDDINGS WITH MiniLM AND COMPARE SIMILARITIES
def compute_minilm_embeddings(chunks, emb_model):
    print("\n=== 3) Compute embeddings with MiniLM and compare similarity scores ===\n")

    embeddings = emb_model.encode(chunks, normalize_embeddings=True)
    sim_matrix = np.matmul(embeddings, embeddings.T)

    print("Example chunks (first 5):")
    for i, ch in enumerate(chunks[:5]):
        print(f"\nChunk {i} (≈{len(ch.split())} words):")
        print(ch)

    print("\nSimilarity matrix (cosine) – first 5x5 block:")
    np.set_printoptions(precision=3, suppress=True)
    n = min(5, len(chunks))
    print(sim_matrix[:n, :n])

    return embeddings, sim_matrix


# 4. IDENTIFY ONE "BAD CHUNK" AND FIX IT
def find_bad_chunk(sim_matrix):
    """Identify the chunk with the lowest average similarity to other chunks."""
    n = sim_matrix.shape[0]
    avg_sims = []
    for i in range(n):
        avg_sim = (sim_matrix[i].sum() - 1.0) / (n - 1)  # exclude self-similarity 1.0
        avg_sims.append(avg_sim)

    bad_index = int(np.argmin(avg_sims))
    return bad_index, avg_sims


def fix_bad_chunk(chunks, sim_matrix):
    print("\n=== 4) Identify one 'bad chunk' and fix it ===\n")

    bad_idx, avg_sims = find_bad_chunk(sim_matrix)

    print("Average similarity per chunk:")
    for i, s in enumerate(avg_sims):
        print(f"Chunk {i}: avg similarity = {s:.3f}")

    print(f"\n👉 Bad chunk candidate: Chunk {bad_idx} (lowest avg similarity)\n")
    print("Bad chunk content:")
    print(chunks[bad_idx], "\n")

    n = len(chunks)
    if n == 1:
        print("Only one chunk, nothing to fix.")
        return chunks

    # Choose neighbor with higher similarity
    if bad_idx == 0:
        neighbor_idx = 1
    elif bad_idx == n - 1:
        neighbor_idx = n - 2
    else:
        sim_prev = sim_matrix[bad_idx, bad_idx - 1]
        sim_next = sim_matrix[bad_idx, bad_idx + 1]
        neighbor_idx = bad_idx - 1 if sim_prev >= sim_next else bad_idx + 1

    print(f"Merging bad chunk {bad_idx} with neighbor chunk {neighbor_idx} to improve coherence.\n")

    # Build merged chunk list
    new_chunks = []
    i = 0
    while i < n:
        if i == bad_idx:
            if neighbor_idx < bad_idx:
                # neighbor comes before bad_idx
                merged_text = chunks[neighbor_idx] + " " + chunks[bad_idx]
                new_chunks.append(merged_text)
                i += 1  # neighbor already processed earlier
            else:
                # neighbor comes after bad_idx
                merged_text = chunks[bad_idx] + " " + chunks[neighbor_idx]
                new_chunks.append(merged_text)
                i += 2  # skip bad_idx and neighbor_idx
        elif i == neighbor_idx:
            # will be merged in bad_idx branch
            i += 1
        else:
            new_chunks.append(chunks[i])
            i += 1

    print("New chunks after fix (first 5 shown):")
    for j, ch in enumerate(new_chunks[:5]):
        print(f"\nChunk {j} (≈{len(ch.split())} words):")
        print(ch)

    return new_chunks


def main():
    # Tokenizer for Albanian (multilingual SentencePiece – mT5)
    sp_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

    # Embedding model (MiniLM)
    emb_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    # 1) Tokenize 10 emergency terms, print tokens, IDs, and embeddings
    tokenize_emergency_terms(sp_tokenizer, emb_model)

    # 2) Load the full Albanian emergency document and chunk it by SENTENCE
    print("\n=== 2) Load full emergency document and chunk by sentences ===\n")
    document = load_emergency_document(DOC_PATH)
    total_words = len(document.split())
    print(f"Full document length: ~{total_words} words")

    chunks = chunk_text_by_sentences(document, max_words_per_chunk=80)
    print(f"Number of sentence-based chunks created: {len(chunks)}")

    # Show first few chunks
    for i, ch in enumerate(chunks[:5]):
        print(f"\nChunk {i} (≈{len(ch.split())} words):")
        print(ch)

    # 3) Compute embeddings with MiniLM and compare similarity scores
    embeddings, sim_matrix = compute_minilm_embeddings(chunks, emb_model)

    # 4) Identify one “bad chunk” and fix it
    fixed_chunks = fix_bad_chunk(chunks, sim_matrix)

    # 5) (Optional) Recompute similarity after fix
    print("\n=== 5) Recompute similarity after fixing the bad chunk (optional) ===\n")
    if fixed_chunks:
        fixed_embeddings = emb_model.encode(fixed_chunks, normalize_embeddings=True)
        fixed_sim_matrix = np.matmul(fixed_embeddings, fixed_embeddings.T)
        n2 = min(5, len(fixed_chunks))
        print("New similarity matrix after fix – first 5x5 block:")
        print(fixed_sim_matrix[:n2, :n2])


if __name__ == "__main__":
    main()



=== 1) Tokenize 10 Albanian emergency terms ===

Term: përmbytje
  Tokens: ['▁për', 'mby', 'tje']
  Token IDs: [1212, 85107, 10281]
  Embedding (first 10 dims): [ 0.017  0.06  -0.065 -0.009 -0.067  0.028  0.1    0.062  0.081 -0.023]

Term: tërmet
  Tokens: ['▁', 'tër', 'met']
  Token IDs: [259, 25741, 4116]
  Embedding (first 10 dims): [ 0.023 -0.014 -0.074 -0.061 -0.062 -0.079  0.14   0.074  0.024  0.048]

Term: zjarr
  Tokens: ['▁z', 'jar', 'r']
  Token IDs: [397, 3879, 286]
  Embedding (first 10 dims): [-0.094  0.113 -0.056 -0.007  0.041 -0.082  0.019  0.049  0.064 -0.022]

Term: evakuim
  Tokens: ['▁evaku', 'im']
  Token IDs: [164337, 780]
  Embedding (first 10 dims): [-0.082  0.073 -0.074  0.068  0.051 -0.082  0.18   0.046 -0.005 -0.024]

Term: ndihmë
  Tokens: ['▁ndi', 'hmë']
  Token IDs: [3025, 159002]
  Embedding (first 10 dims): [-0.022  0.078 -0.047  0.01   0.    -0.005  0.073  0.02   0.044 -0.033]

Term: ambulancë
  Tokens: ['▁ambulan', 'c', 'ë']
  Token IDs: [88205, 297, 7